<a href="https://colab.research.google.com/github/AI-Isaiah/AI-Isaiah/blob/main/20230329_9_ensemble_weighted_method_with_smote_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import ElasticNetCV
from sklearn.svm import SVC
!pip install optuna
!pip install catboost
#!pip install verstack
#from verstack import LGBMTuner
import warnings
import seaborn as sns
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping, log_evaluation
from sklearn.pipeline import Pipeline
import sys
import lightgbm as lgbm
import logging
sys.path.append('/bin/sample_data')
sys.path.append('/root')
#from preprocessor import generate_cm
from collections import Counter
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
import optuna
import pickle
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, RandomizedSearchCV, ParameterSampler, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, precision_score, log_loss, f1_score, roc_auc_score, roc_curve, recall_score, accuracy_score, make_scorer,ConfusionMatrixDisplay,matthews_corrcoef, classification_report
from sklearn.feature_selection import RFECV
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import gc
import json
from imblearn.over_sampling import SMOTE, ADASYN, SMOTENC


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.7 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
def generate_cm1(y_true, y_pred, title):
    y_pred = (y_pred > 0.5).astype(int)
    cm = confusion_matrix(y_true, y_pred)

    # Display confusion matrix using ConfusionMatrixDisplay
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Class 0", "Class 1"])
    disp.plot()
    counter = Counter(y_true)
    for key, value in counter.items():
      print(f'{title}"{key}": {value}')

class NumericalScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        self.num_cols = []

    def fit(self, X, y=None):
        self.num_cols = X.select_dtypes(include=['int64', 'float64']).columns
        self.scaler.fit(X[self.num_cols])
        return self

    def transform(self, X):
        X_scaled = X.copy()
        X_scaled[self.num_cols] = self.scaler.transform(X[self.num_cols])
        return X_scaled

class FeatureEliminator:
    def __init__(self, X_train, y_train, X_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test

    @staticmethod
    def objective(trial, X, y):
        xgb_fe_params = {
            "tree_method": "hist",
            'objective': 'binary:logistic',
            "eval_metric": "logloss",
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 100, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 100, log=True),
        }

        dtrain = xgb.DMatrix(X, label=y, enable_categorical=True, eval_metric = "logloss")
        num_boost_round = trial.suggest_int('num_boost_round', 50, 1000)
        #model = xgb.train(params, dtrain, num_boost_round = num_boost_round)
        #y_pred = model.predict(dtrain)
        cv_result = xgb.cv(xgb_fe_params, 
                       dtrain, nfold=5, 
                       num_boost_round = num_boost_round, 
                       stratified=True, 
                       early_stopping_rounds=50, 
                       seed = 42, 
                       maximize = False)
    
        min_logloss_val = cv_result["test-logloss-mean"].min()
    
        return min_logloss_val
        
    def eliminate_features(self, correlated_features):
        # Remove the least important feature from correlated pairs
        for feature_pair in correlated_features:
            study = optuna.create_study(direction='minimize')
            study.optimize(lambda trial: self.objective(trial, self.X_train[[feature_pair[0]]].values, self.y_train), n_trials=50)
            score_0 = study.best_value

            study = optuna.create_study(direction='minimize')
            study.optimize(lambda trial: self.objective(trial, self.X_train[[feature_pair[1]]].values, self.y_train), n_trials=50)
            score_1 = study.best_value

            if score_0 < score_1:
                self.X_train.drop(feature_pair[1], axis=1, inplace=True)
            else:
                self.X_train.drop(feature_pair[0], axis=1, inplace=True)

        # Update X_train and X_test after removing correlated features
        self.X_test = pd.DataFrame(self.X_test, columns=self.X_train.columns)

def find_correlated_features(df, threshold=0.9):
    correlated_features = set()
    correlation_matrix = df.corr()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                colname_i = correlation_matrix.columns[i]
                colname_j = correlation_matrix.columns[j]
                correlated_features.add((colname_i, colname_j))

    return list(correlated_features)



In [4]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/loan_book_data_labels.csv")
labels = data["Result"]
data.drop("Result", axis=1, inplace=True)
print(labels.head())
print(data.head())

FileNotFoundError: ignored

In [ ]:
X = data
if "appl_ref" in data.columns:
    # drop the column
    data.drop("appl_ref", axis=1, inplace=True)

for i in range(52, 70):
  column_name = "v" + str(i) + "_src"
  if column_name in data.columns:
    data.drop(column_name, axis=1, inplace=True)

X = data
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
print(X.dtypes)
feature_names = X.columns
le = LabelEncoder()
y = le.fit_transform(labels)
y = y.ravel()
cat_columns = X.select_dtypes(include='category').columns

num_columns = X.select_dtypes(exclude=['category']).columns

# Encode categorical columns using LabelEncoder
"""label_encoders = {}
for col in cat_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le"""



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train = pd.DataFrame(X_train, columns=X.columns)
y_train = pd.DataFrame(y_train, columns=["Result"])
X_test = pd.DataFrame(X_test, columns=X.columns)
y_test = pd.DataFrame(y_test, columns=["Result"])

#Get the correlated foreatures from X_train
correlated_features = find_correlated_features(X_train, threshold=0.8)

#instantiate class FeatureEliminator
fe = FeatureEliminator(X_train, y_train, X_test)

# Call the eliminate_features method with the correlated_features list
fe.eliminate_features(correlated_features)

# Get the updated X_train and X_test
X_train = fe.X_train
X_test = fe.X_test


# Identify categorical columns
cat_cols = X_train.select_dtypes(include=['category']).columns.tolist()
print("Cat-cols: ", cat_cols)
num_cols = X_train.select_dtypes(exclude=['category']).columns.tolist()
# Create a SMOTENC object with categorical features
smote_nc = SMOTENC(categorical_features=[X_train.columns.get_loc(c) for c in cat_cols], random_state=42)

# Fit the SMOTENC object to the training data
X_train, y_train = smote_nc.fit_resample(X_train, y_train)

print("X_train_shape final: ", X_train.shape)
print("X_train dtypes:" , X_train.dtypes)


In [ ]:
preprocessor = NumericalScaler()

# Fit the preprocessor on the training data
X_train= preprocessor.fit_transform(X_train)

# Apply the preprocessor on the validation and test data
X_test = preprocessor.transform(X_test)



# Random Forest Classifier 


1.   Optuna - XXX algorithm 
2.   cross_val with StratKFold
3. Scoring Neg Log loss






In [ ]:
 #Define the objective function for Optuna
def objective(trial, X, y):
    "n_estimators" : trial.suggest_int("n_estimators", 10, 2000)
    "criterion" : trial.suggest_categorical("criterion", ["gini", "entropy", "mse", "mae"])
    "max_depth" : trial.suggest_int("max_depth", 2, 32, log=True)
    "min_samples_split" : trial.suggest_int("min_samples_split", 2, 20)
    "min_samples_leaf" : trial.suggest_int("min_samples_leaf", 1, 20)
    "max_features" : trial.suggest_categorical("max_features", ["auto", "sqrt", "log2"])
    "bootstrap" : True,
    "class_weight" : trial.suggest_categorical("class_weight", [None, "balanced", "balanced_subsample"])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        class_weight=class_weight,
        random_state=42,
        n_jobs=-1,
    )

    # Perform cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring="neg_log_loss")
    mean_score = np.mean(scores)

    return mean_score

# Optimize hyperparameters with Optuna
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.NSGAIISampler(seed=42))
study.optimize(lambda trial: objective(trial, X, y), n_trials=100, timeout=600)

# Print best trial
print("Best trial:")
trial = study.best_trial
print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"{key}: {value}")


# ElasticNetCV 


1.   Simple native algorithm
2. One Hot encoding




In [ ]:
# One-hot encoding of selected categorical features
X_train_ohe = pd.get_dummies(X_train, columns=cat_cols)
X_val_ohe = pd.get_dummies(X_val, columns=cat_cols)
X_val_ohe = pd.get_dummies(X_test, columns=cat_cols)


ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
                                n_alphas=100,
                                cv=10,
                                max_iter=10000,
                                random_state=42,
                                n_jobs=-1))
])
# ElasticNetCV model
l1_ratios = np.linspace(.01,.05, .1, .2, .3, .4, .5, .6, .7, .9, .95, .99, 1)
model = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=100, cv=10, random_state=42, n_jobs=-1)
model.fit(X_train_ohe, y_train_ohe)

# Prediction and evaluation
y_pred = model.predict(X_val_ohe)
mse = mean_squared_error(y_val, y_pred)

print(f"Optimal l1_ratio: {model.l1_ratio_}")
print(f"Optimal alpha: {model.alpha_}")
print(f"Mean squared error: {mse}")




# LGBM Classifier 


1.   Optuna - XXX algorithm 
2.   cross_val with StratKFold
3. Scoring Neg Log loss
4. 



In [ ]:
def objective_lgbm(trial, X, y):
    
    lgbm_params = {
      'objective': 'binary',
      'boosting_type': 'gbdt',
      "metric": "binary_logloss",
      'max_depth': trial.suggest_int('max_depth', 5, 20),
      'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3),
      'num_leaves': trial.suggest_int('num_leaves', 20, 80),
      'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
      'min_child_weight': trial.suggest_float('min_child_weight', 1e-5, 100),
      'subsample': trial.suggest_float('subsample', 0.1, 1.0, step=0.1),
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
      'reg_alpha': trial.suggest_float('reg_alpha', 0, 10, step=0.1),
      'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
      'min_split_gain': trial.suggest_float('min_split_gain', 0, 1),
      'max_bin': trial.suggest_int('max_bin', 300, 800),
      'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
      'verbosity': -1,
      'seed': 42
    }


    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    num_boost_round = trial.suggest_int('num_boost_round', 50, 1000)
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        lgbm_train = lgbm.Dataset(X_train, y_train, categorical_feature=cat_cols)
        lgbm_valid = lgbm.Dataset(X_test, y_test, categorical_feature=cat_cols)#, reference=lgbm_train)
        
        #model = lgbm.LGBMClassifier(**lgbm_params)
        model = lgbm.train(lgbm_params, 
                           lgbm_train,
                           num_boost_round = num_boost_round,
                           valid_sets=[lgbm_valid],
                           callbacks=[early_stopping(stopping_rounds=100, first_metric_only=False),
                                      log_evaluation(period=10)],
        )
        preds = model.predict(X_test)
        cv_scores[idx] = log_loss(y_test, preds)
    print(f'Trial {trial.number}: Parameters: {lgbm_params}, Logloss: {np.mean(cv_scores)}')
    return -np.mean(cv_scores)
    
starting_lgbm_hp = {
  
}
#optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
#optuna.logging.set_verbosity(10)
study = optuna.create_study(direction='maximize', 
                            sampler=optuna.samplers.NSGAIISampler(seed=42), 
                            pruner=optuna.pruners.HyperbandPruner(),
                            )

warnings.filterwarnings("ignore", category=UserWarning)
study_logger = logging.getLogger("optuna")
study_logger.setLevel(logging.WARNING)
study_logger.addHandler(logging.StreamHandler())

study.enqueue_trial(starting_lgbm_hp)
study.optimize(lambda trial: objective_lgbm(trial, X_train, y_train), 
               n_trials=18000, 
               show_progress_bar=True)

best_params = study.best_params
print(f"Best log loss: {study.best_value:.4f}")
print(f"Best params: {best_params}")

lgbm_train = lgbm.Dataset(X_train, y_train, categorical_feature=cat_cols)
lgbm_val = lgbm.Dataset(X_val, y_val, categorical_feature=cat_cols)#, reference=lgbm_train)
lgbm_test = lgbm.Dataset(X_test, categorical_feature=cat_cols)#, reference=lgbm_val)

lgbm_clf = lgbm.train(best_params, lgbm_train)
y_pred = lgbm_clf.predict(X_test)

y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred]
print("y_pred_binary_len: ", len(y_pred_binary))
print("y_test shape: ", y_test.shape)
accuracy = sum(y_pred_binary == y_test.squeeze()) / len(y_test)
print(f"Accuracy on test set: {accuracy:.4f}")


lgbm_best_trial = study.best_trial
lgbm_best_params = lgbm_best_trial.params

print(f"Best score: {lgbm_best_trial.value:.4f}")
print(f"Best params: {lgbm_best_params}")

with open('trial_lgbm.pickle', 'wb') as f:
    pickle.dump(lgbm_best_params, f)


generate_cm1(y_val, lgbm_clf.predict(X_val), "XGBoost, optuna - cv, Validation Set")
generate_cm1(y_test, lgbm_clf.predict(X_test), "XGBoost, optuna - cv, Validation Set")


print("Counter - valid" , y_val.value_counts())
print("Counter - test", y_test.value_counts())


with open('lgbm_best_params.json', 'w') as f:
    json.dump(lgbm_best_params, f)



In [ ]:
def custom_loss_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    y_pred = (y_pred > 0.5).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    custom_metric = (tn * 4) - (fn * 10)
    return 'custom_loss', custom_metric

def custom_obj(y_true, y_pred):
    grad = (y_pred - y_true)
    hess = np.ones_like(y_true)
    fn_mask = y_true == 0  # identify false negatives
    grad[fn_mask] *= 3  # triple-penalize false negatives
    hess[fn_mask] *= 2  # increase penalty for false negatives
    tn_mask = y_true == 1  # identify true negatives
    grad[tn_mask] += 2  # give bonus for true negatives
    hess[tn_mask] *= 2  # increase bonus for true negatives
    return grad, hess

class CustomMetric:
    def __call__(self, y_true, y_pred):
        y_pred = (y_pred > 0.5).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        custom_metric = (tn * 4) - (fn * 10)
        return 'custom_metric', custom_metric
custom_metric = CustomMetric()

In [ ]:
def custom_obj(y_true, y_pred):
    y_pred = y_pred.get_label()
    grad = y_pred - y_true
    hess = np.ones_like(y_true)

    y_pred_labels = (y_pred > 0.5).astype(int)
    fn_mask = (y_true == 1) & (y_pred_labels == 0)  # identify false negatives
    grad[fn_mask] -= 3  # triple-penalize false negatives
    hess[fn_mask] *= 2  # increase penalty for false negatives

    tn_mask = (y_true == 0) & (y_pred_labels == 0)  # identify true negatives
    grad[tn_mask] += 2  # give bonus for true negatives
    hess[tn_mask] *= 2  # increase bonus for true negatives

    return grad, hess
class CustomMetric:
    def __call__(self, y_true, y_pred):
        y_pred = y_pred.get_label()
        y_pred = (y_pred > 0.5).astype(int)
        y_true = (y_true > 0.5).astype(int)

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        custom_metric = (tn * 4) - (fp * 4)
        return 'custom_metric', custom_metric
custom_metric = CustomMetric()

class CustomMetric2:
  def custom_eval(preds, dmatrix):
      labels = dmatrix.get_label()
      preds_binary = (preds > 0.5).astype(int)
      mcc = matthews_corrcoef(labels, preds_binary)
      fn_mask = labels == 1  # identify false negatives
      tn_mask = labels == 0  # identify true negatives
      mcc_fn = mcc * (1 - sum(fn_mask) / len(labels))  # multiply MCC by the fraction of true negatives
      mcc_tn = mcc * sum(tn_mask) / len(labels)  # multiply MCC by the fraction of false negatives
      return "mcc", (mcc_fn + mcc_tn) / 2, True
custom_metric2 = CustomMetric2()

def objective_xgb(trial,study, X, y):
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    #dtrain = xgb.DMatrix(X_train, label=y_train)
    #dtest = xgb.DMatrix(X_test, label=y_test)
    #dtrain = xgb.DMatrix(X, label=y)
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.1, 0.6),
        "tree_method": "hist",
        'objective': 'binary:logistic',
        "eval_metric": "logloss",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 0.4),
        "alpha": trial.suggest_float("alpha", 1e-8, 0.6),
        "reg_alpha": trial.suggest_float("alpha", 1e-8, 0.6),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 0.4),
        "verbosity": 0
    }

    if xgb_params["booster"] == "gbtree" or xgb_params["booster"] == "dart":
        xgb_params["max_depth"] = trial.suggest_int("max_depth", 1, 16)
        xgb_params["eta"] = trial.suggest_float("eta", 1e-8, 0.6)
        xgb_params["gamma"] = trial.suggest_float("gamma", 1e-8, 0.6)
        xgb_params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if xgb_params["booster"] == "dart":
        xgb_params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        xgb_params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        xgb_params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 0.6)
        xgb_params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 0.6)

    #pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    #pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'logloss')
    #bst = xgb.train(xgb_params, dtrain, evals=[(dtest, "validation")], callbacks=[pruning_callback])
    dtrain = xgb.DMatrix(X, label=y, enable_categorical = True)
    dval = xgb.DMatrix(X_val, label=y_val, enable_categorical = True)
    evals = [(dtrain, 'train'), (dval, 'val')]
    num_boost_round = trial.suggest_int('num_boost_round', 50, 1000)
    cv_result = xgb.cv(xgb_params, 
                       dtrain, nfold=5, 
                       num_boost_round = num_boost_round, 
                       stratified=True, 
                       early_stopping_rounds=50, 
                       seed = 42, 
                       maximize = False)
    
    logloss_train = cv_result["train-logloss-mean"].iloc[-1]
    logloss_val = cv_result["test-logloss-mean"].iloc[-1]

    min_logloss_val = cv_result["test-logloss-mean"].min()
    
    return -min_logloss_val
   
    #logloss = cv_result["test-logloss-mean"].min()
    #return logloss
    """
    mean_auc = cv_result['test-auc-mean'].max()
    #mean_error = cv_result["test-error-mean"].min()
    #return 1.0 - mean_error
    #y_preds = bst.predict(dtest)
    #y_preds_labels = np.rint(y_preds)
    #accuracy = accuracy_score(y_test, y_preds_labels)
    #return accuracy
    #return mean_auc
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    num_boost_round = trial.suggest_int('num_boost_round', 50, 500)
    # Perform cross-validation with the custom objective and evaluation metric
    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
        dval = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

        # Create a dictionary to store the evaluation results
        evals_result = {}

        # Train the model using the custom objective and evaluation metric
        bst = xgb.train(
            xgb_params,
            dtrain,
            num_boost_round=num_boost_round,
            evals=[(dtrain, 'train'), (dval, 'val')],
            obj="binary:logistic"
            custom_metric="accuracy"
            evals_result=evals_result,
            early_stopping_rounds=10,
            maximize=True,
            verbose_eval=False
        )

        # Get the best validation score
        best_val_score = max(evals_result['val']['custom_metric'])
        scores.append(best_val_score)

    # Calculate the mean validation score
    mean_val_score = np.mean(scores)

    return mean_val_score"""

starting_xgb_hp = {
    'learning_rate': 0.07937792496092369, 
    'min_child_weight': 1, 
    'subsample': 0.5072873601156911, 
    'booster': 'gbtree', 
    'lambda': 0.2101881276876542, 
    'alpha': 0.1022775152443601, 
    'reg_lambda': 0.15380633163734847, 
    'max_depth': 6, 
    'eta': 0.25908426274659535, 
    'gamma': 0.46906789773943747, 
    'grow_policy': 'lossguide', 
    'num_boost_round': 368
}
#optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
#optuna.logging.set_verbosity(10)
study = optuna.create_study(direction='maximize', 
                            sampler=optuna.samplers.NSGAIISampler(seed=42), 
                            pruner=optuna.pruners.HyperbandPruner(min_resource=1, 
                                                                  max_resource="auto", 
                                                                  reduction_factor=3),
                            )
study_logger = logging.getLogger("optuna")
study_logger.setLevel(1)
study_logger.addHandler(logging.StreamHandler())

study.enqueue_trial(starting_xgb_hp)
study.optimize(lambda trial: objective_xgb(trial, study, X_train, y_train), 
               n_trials=600, 
               show_progress_bar=True)


xgb_best_trial = study.best_trial
xgb_best_params = xgb_best_trial.params

print(f"Best score: {xgb_best_trial.value:.4f}")
print(f"Best params: {xgb_best_params}")

with open('trial_xgb.pickle', 'wb') as f:
    pickle.dump(xgb_best_params, f)


#xgb_clf = XGBClassifier(**xgb_best_params, tree_method="gpu_hist", enable_categorical=True, max_to_one_hot=1)
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical = True)
dval = xgb.DMatrix(X_val, label=y_val, enable_categorical = True)
evals = [(dtrain, 'train'), (dval, 'val')]
xgb_clf = xgb.train(xgb_best_params, dtrain, num_boost_round=1000, early_stopping_rounds=20, evals=evals)

dX_val =  xgb.DMatrix(X_val, enable_categorical = True)
dX_test = xgb.DMatrix(X_test, enable_categorical = True)
generate_cm1(y_val, xgb_clf.predict(dX_val), "XGBoost, optuna - cv, Validation Set")
generate_cm1(y_test, xgb_clf.predict(dX_test), "XGBoost, optuna - cv, Validation Set")

print("Counter - valid" , y_val.value_counts())
print("Counter - test", y_test.value_counts())



In [ ]:
y_pred = xgb_clf.predict(dX_val)
y_pred = (y_pred > 0.5).astype(int)
tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
print("confusion Matrix validation")
print("tn: ", tn, "fp: ", fp, "fn: ", fn, "tp: ", tp)

# Catboost classifier

In [ ]:
def custom_loss(y_pred, y_true):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return (tn * 4) - (fn * 10)

# Define the objective function for Optuna
def objective_catboost(trial, X, y, cat_cols):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'depth': trial.suggest_int('depth', 2, 10),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 5, 200),
        'random_strength': trial.suggest_int('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.00),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 1, 10),
        'task_type': 'GPU'
    }
    
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Train and evaluate the model using cross-validation
    scores = []
    for train_idx, val_idx in skf.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        # Create a Pool object with categorical features
        pool_train = Pool(X_train, y_train, cat_features=cat_cols)
        pool_valid = Pool(X_val, y_val, cat_features=cat_cols)
        
        # Train the model
        model = CatBoostClassifier(**params, random_state = 42)
        model.fit(pool_train, eval_set=pool_valid, verbose=False)
        
        # Evaluate the model on the validation set
        y_pred = model.predict(X_val)
        y_pred = (y_pred > 0.5).astype(int)
        score = custom_loss(y_val, y_pred)
        scores.append(score)

    return np.mean(scores)

#Starting hyperparamters to improve convergence
starting_cat_hp = {'iterations': 109, 
                            'learning_rate': 0.45363716844543917, 
                            'depth': 2, 
                            'l2_leaf_reg': 4, 
                            'border_count': 191, 
                            'random_strength': 20, 
                            'bagging_temperature': 57.34805443344737, 
                            'grow_policy': 'SymmetricTree', 
                            'max_ctr_complexity': 4
}

# Optimize hyperparameters

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.NSGAIISampler(seed = 42))
study.enqueue_trial(starting_cat_hp)
study.optimize(lambda trial: objective_catboost(trial, X_train,y_train, cat_cols), n_trials=100)

# Get best parameters
best_params = study.best_params
print("Best parameters: ", best_params)

#Pool variables to be used with catboost:
pool_train = Pool(X_train, y_train, cat_features = cat_cols)
pool_valid = Pool(X_val, cat_features = cat_cols)
pool_test = Pool(X_test, cat_features = cat_cols)


# Run the randomized search
# best parameters: iterations': 420, 'learning_rate': 0.18951730321390894, 'depth': 2, 'l2_leaf_reg': 2, 'border_count': 44, 'random_strength': 11, 'bagging_temperature': 59.24553274051562, 'grow_policy': 'SymmetricTree', 'max_ctr_complexity': 5
cb_classifier = CatBoostClassifier(**best_params, random_state=42, verbose=0)
cb_classifier.fit(pool_train)
tn, fp, fn, tp = confusion_matrix(y_val, cb_classifier.predict(X_val)).ravel()
print("confusion Matrix validation")
print("tn: ", tn, "fp: ", fp, "fn: ", fn, "tp: ", tp)
generate_cm(y_val, cb_classifier.predict(pool_valid), "CatBoost - Validation")
tn, fp, fn, tp = confusion_matrix(y_test, cb_classifier.predict(X_test)).ravel()
print("confusion Matrix test")
print("tn: ", tn, "fp: ", fp, "fn: ", fn, "tp: ", tp)
generate_cm(y_test, cb_classifier.predict(pool_test), "CatBoost - Test")

# Save the best parameters for CatBoost
with open('cat_best_params.json', 'w') as f:
    json.dump(study.best_params, f)

# Clear memory
del cb_classifier
gc.collect()

In [ ]:

train_eval_metric = evals_result['train']['custom_metric']
test_eval_metric = evals_result['test']['custom_metric']

# Visualize the evaluation metric
plt.figure(figsize=(10, 6))
plt.plot(train_eval_metric, label='Train')
plt.plot(test_eval_metric, label='Test')
plt.xlabel('Boosting round')
plt.ylabel('Custom metric')
plt.title('Evaluation metric during training process')
plt.legend()
plt.show()

In [ ]:
from sklearn.ensemble import VotingClassifier

# Load the best parameters for each classifier
with open('lgbm_best_params.pkl', 'rb') as f:
lgbm_best_params = pickle.load(f)

with open("trial_xgb.pkl", "rb") as f:
xgb_trial = pickle.load(f)

with open('cat_best_params.pkl', 'rb') as f:
cat_best_params = pickle.load(f)

with open('logreg_best_params.pkl', 'rb') as f:
logreg_best_params = pickle.load(f)

with open('rf_best_params.pkl', 'rb') as f:
rf_best_params = pickle.load(f)

with open('svm_best_params.pkl', 'rb') as f:
svm_best_params = pickle.load(f)




In [ ]:
# Instantiate classifiers with the best parameters
lgbm_clf = LGBMClassifier(random_state=42, **lgbm_best_params)
xgb_clf = XGBClassifier(use_label_encoder=False, random_state=42, tree_method='hist', n_jobs=-1, **xgb_best_params)
cat_clf = CatBoostClassifier(task_type="CPU", random_seed=42, verbose = False, **cat_best_params)
rf_clf = RandomForestClassifier(random_state=42, **rf_best_params)
ela_clf = 

# Create a soft-voting classifier
ensemble_clf = VotingClassifier(estimators=[
    ('lgbm', lgbm_clf),
    ('xgb', xgb_clf),
    ('cat', cat_clf),
    ('logreg', logreg_clf),
    ('rf', rf_clf),
    ('svm', svm_clf)
], voting='soft')



# Train the ensemble classifier
ensemble_clf.fit(X_train, y_train)

# Calculate the recall score for the ensemble
ensemble_recall = recall_score(y_true=y_test, y_pred=ensemble_clf.predict(X_test), pos_label=1)
print(f"Ensemble recall score: {ensemble_recall}")

generate_cm(y_val, ensemble_clf.predict(X_val))
generate_cm(y_test, ensemble_clf.predict(X_test))


In [ ]:
y_pred = ensemble_clf.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

print("Confusion Matrix:")
print(cm)


In [ ]:
y_pred = ensemble_clf.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

print("Confusion Matrix:")
print(cm)

In [ ]:
precision = precision_score(y_true=y_test, y_pred=y_pred, pos_label=1)
recall = recall_score(y_true=y_test, y_pred=y_pred, pos_label=1)
f1 = f1_score(y_true=y_test, y_pred=y_pred, pos_label=1)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


In [ ]:
y_pred_proba = ensemble_clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)

fpr, tpr, _ = roc_curve(y_true=y_test, y_score=y_pred_proba)

plt.figure()
plt.plot(fpr, tpr, label=f"Ensemble Classifier (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


Addition of feauture reduction methods

In [ ]:

import os
from contextlib import redirect_stdout

ensemble_clf.fit(X_train, y_train)

def voting_classifier_feature_importances(voting_clf):
    importances = []
    for clf in voting_clf.estimators_:
        if isinstance(clf, tuple):  # Check if the clf is a tuple (name, estimator)
            clf = clf[1]
        if hasattr(clf, 'coef_'):
            importances.append(clf.coef_.flatten())
        elif hasattr(clf, 'feature_importances_'):
            importances.append(clf.feature_importances_)
    
    if not importances:
        raise ValueError("No estimators in the ensemble have `coef_` or `feature_importances_` attribute.")
    
    return np.mean(importances, axis=0)


rfe = RFECV(estimator=ensemble_clf, step=1, cv=5, scoring='recall', n_jobs=-1, importance_getter=voting_classifier_feature_importances)

with open(os.devnull, 'w') as devnull:
    with redirect_stdout(devnull):
        rfe.fit(X_train, y_train)

# Print the optimal number of features
print("Optimal number of features:", rfe.n_features_)

In [ ]:
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)
# Print the optimal number of features
print("Optimal number of features:", rfe.n_features_)

In [ ]:
ensemble_clf.fit(X_train_rfe, y_train)
y_pred_rfe = ensemble_clf.predict(X_test_rfe)

accuracy_rfe = accuracy_score(y_test, y_pred_rfe)
report_rfe = classification_report(y_test, y_pred_rfe)
conf_matrix_rfe = confusion_matrix(y_test, y_pred_rfe)

print(f"Accuracy (with RFE): {accuracy_rfe:.4f}")
print("Classification Report (with RFE):")
print(report_rfe)
print("Confusion Matrix (with RFE):")
print(conf_matrix_rfe)


Use stacking method to combine the results of different models
Extra trees classifier and Principal component analysis

In [ ]:
ex_clf = ExtraTreesClassifier(random_state=42)

pipeline = Pipeline([
    ('pca', PCA()),
    ('ex_clf', ex_clf),
])
param_grid = {
    'pca__n_components': np.arange(1, X_train.shape[1] + 1),
    'ex_clf__n_estimators': [100, 200, 500],
    'ex_clf__max_depth': [None, 10, 20, 30],
    'ex_clf__min_samples_split': [2, 5, 10],
    'ex_clf__min_samples_leaf': [1, 2, 4],
}

random_search = RandomizedSearchCV(pipeline, param_grid, n_iter=50, cv=5, scoring='recall', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)
ex_clf_best_params = random_search.best_params_

y_pred = random_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(range(X_train.shape[1]), importances[sorted_indices])
plt.xticks(range(X_train.shape[1]), sorted_indices)
plt.xlabel("Feature index")
plt.ylabel("Importance")
plt.title("Feature importances in the best ExtraTreesClassifier model")
plt.show()


Use the other classifiers to predict the test data

In [ ]:
# Instantiate classifiers

lgbm_clf = LGBMClassifier(random_state=42, **lgbm_best_params)
xgb_clf = XGBClassifier(use_label_encoder=False, random_state=42, tree_method='hist', n_jobs=-1, **xgb_best_params)
cat_clf = CatBoostClassifier(task_type="CPU", random_seed=42, **cat_best_params)
rf_clf = RandomForestClassifier(random_state=42, **rf_best_params)
ElastinNetCV

# Create pipelines with PCA and classifiers
pipelines = {
    
    'LightGBM': Pipeline([('pca', PCA()), ('clf', lgbm_clf)]),
    'XGBoost': Pipeline([('pca', PCA()), ('clf', xgb_clf)]),
    'CatBoost': Pipeline([('pca', PCA()), ('clf', cat_clf)]),
    'Logistic Regression': Pipeline([('pca', PCA()), ('clf', logreg_clf)]),
    'Random Forest': Pipeline([('pca', PCA()), ('clf', rf_clf)]),
    'SVM': Pipeline([('pca', PCA()), ('clf', svm_clf)]),
}



In [ ]:
cv_results = []

for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall', n_jobs=-1)
    cv_results.append((name, scores.mean(), scores.std()))

cv_results.sort(key=lambda x: x[1], reverse=True)

print("Classifier performance using cross-validation:")
for name, mean_score, std_dev in cv_results:
    print(f"{name}: {mean_score:.4f} (+/- {std_dev:.4f})")



Stacking classifier of PCA analysis

In [ ]:
base_classifiers = [
    
    ('LightGBM', lgbm_clf),
    ('XGBoost', xgb_clf),
    ('CatBoost', cat_clf),
    ('Logistic Regression', logreg_clf),
    ('Random Forest', rf_clf),
    ('SVM', svm_clf),
    
]

meta_classifier = XGBClassifier(random_state=42)
stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier)


In [ ]:
stacking_scores = cross_val_score(stacking_clf, X_train, y_train, cv=5, scoring='recall', n_jobs=-1)
print("Stacking Classifier performance:")
print(f"Mean recall: {stacking_scores.mean():.4f}, Standard deviation: {stacking_scores.std():.4f}")


In [ ]:
stacking_clf.fit(X_train, y_train)
y_pred = stacking_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)
generate_cm(y_val, stacking_clf.predict(X_val), "Stacking Classifier")
generate_cm(y_test, stacking_clf.predict(X_test), "Stacking Classifier")


In [ ]:
class OptunaWeights:
    def __init__(self, random_state):
        self.study = None
        self.weights = None
        self.random_state = random_state

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))]

        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=weights)

        # Calculate the AUC score for the weighted prediction
        score = roc_auc_score(y_true, weighted_pred)
        return score

    def fit(self, y_true, y_preds, n_trials=2000):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        self.study = optuna.create_study(sampler=sampler, study_name="OptunaWeights", direction='maximize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds, n_trials=2000):
        self.fit(y_true, y_preds, n_trials=n_trials)
        return self.predict(y_preds)
    
    def weights(self):
        return self.weights

# Train weighted model

In [ ]:
kfold = True
n_splits = 1 if not kfold else 10
random_state = 2023
random_state_list = [2140] # used by split_data [71]
n_estimators = 9999 # 9999
early_stopping_rounds = 200
verbose = False
device = 'cpu'

splitter = Splitter(kfold=kfold, n_splits=n_splits)

# Initialize an array for storing test predictions
test_predss = np.zeros(X_test.shape[0])
ensemble_score = []
weights = []
trained_models = {'xgb':[], 'lgb':[], 'cat':[]}

    
for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, y_train, random_state_list=random_state_list)):
    n = i % n_splits
    m = i // n_splits
            
    # Get a set of Regressor models
    classifier = Classifier(n_estimators, device, random_state)
    models = classifier.models
    
    # Initialize lists to store oof and test predictions for each base model
    oof_preds = []
    test_preds = []
    
    # Loop over each base model and fit it to the training data, evaluate on validation data, and store predictions
    for name, model in models.items():
        if name in ['xgb', 'lgb', 'cat']:
            model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], early_stopping_rounds=early_stopping_rounds, verbose=verbose)
            trained_models[f'{name}'].append(deepcopy(model))
        else:
            model.fit(X_train_, y_train_)
        
        test_pred = model.predict_proba(X_test)[:, 1]
        y_val_pred = model.predict_proba(X_val)[:, 1]

        score = roc_auc_score(y_val, y_val_pred)
        print(f'{name} [FOLD-{n} SEED-{random_state_list[m]}] AUC score: {score:.5f}')
        
        oof_preds.append(y_val_pred)
        test_preds.append(test_pred)
    
    # Use Optuna to find the best ensemble weights
    optweights = OptunaWeights(random_state=random_state)
    y_val_pred = optweights.fit_predict(y_val.values, oof_preds)
    
    score = roc_auc_score(y_val, y_val_pred)
    print(f'Ensemble [FOLD-{n} SEED-{random_state_list[m]}] AUC score {score:.5f}')
    ensemble_score.append(score)
    weights.append(optweights.weights)
    
    test_predss += optweights.predict(test_preds) / (n_splits * len(random_state_list))
    
    gc.collect()

In [ ]:
# Calculate the mean LogLoss score of the ensemble
mean_score = np.mean(ensemble_score)
std_score = np.std(ensemble_score)
print(f'Ensemble Accuracy score {mean_score:.5f} ± {std_score:.5f}')

# Print the mean and standard deviation of the ensemble weights for each model
print('--- Model Weights ---')
mean_weights = np.mean(weights, axis=0)
std_weights = np.std(weights, axis=0)
for name, mean_weight, std_weight in zip(models.keys(), mean_weights, std_weights):
    print(f'{name}: {mean_weight:.5f} ± {std_weight:.5f}')